# Predict Consumer Electronics Sales Dataset

**Veri seti içeriği:**<br>
Bu veri seti, ürün kategorileri, markalar, fiyatlar, müşteri demografisi, satın alma davranışı ve memnuniyet ölçümlerini içeren tüketici elektroniği satışlarına ilişkin bilgiler sağlar. Tüketici elektroniği pazarında satın alma niyetini ve müşteri memnuniyetini etkileyen faktörleri analiz etmeyi amaçlamaktadır.

**Veri setinin Özellikleri:**<br>
**ProductID:** Her ürün için benzersiz tanımlayıcı. <br>
**ProductCategory:** Tüketici elektroniği ürün kategorisi (örn. Akıllı Telefonlar, Dizüstü Bilgisayarlar).<br>
**ProductBrand:** Ürünlerin Markası (e.g., Apple, Samsung).<br>
**ProductPrice:** Ürünlerin satış fiyatı ($).<br>
**CustomerAge:** Müşterilerin yaşı.<br>
**CustomerGender:** Müşterilerin cinsiyeti (0 - Erkek, 1 - Kadın).<br>
**PurchaseFrequency:** Yıl içinde ortalama alış miktarları.<br>
**CustomerSatisfaction:** Müşterilerin memnuniyet ratingleri (1 - 5).<br>
**PurchaseIntent (Target Variable):** Satın alma niyeti.<br>

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [77]:
def load():
    df = pd.read_csv('consumer_electronics_sales_data.csv')
    return df

In [78]:
df = load()

In [79]:
def checkdf(data):
    print(data.head(),'\n\n')
    print(data.tail(),'\n\n')
    print(data.shape,'\n\n')
    print(data.info(),'\n\n')
    print(data.describe().T,'\n\n')
    print(data.isnull().sum(),'\n\n')

In [80]:
checkdf(df)

   ProductID ProductCategory  ProductBrand  ProductPrice  CustomerAge  \
0       5874     Smartphones  Other Brands    312.949668           18   
1       5875   Smart Watches       Samsung    980.389404           35   
2       5876         Tablets       Samsung   2606.718293           63   
3       5877     Smartphones       Samsung    870.395450           63   
4       5878         Tablets          Sony   1798.955875           57   

   CustomerGender  PurchaseFrequency  CustomerSatisfaction  PurchaseIntent  
0               0                  2                     1               0  
1               1                  7                     2               1  
2               0                  1                     5               1  
3               1                 10                     3               1  
4               0                 17                     3               0   


      ProductID ProductCategory ProductBrand  ProductPrice  CustomerAge  \
8995      14869   Sma

In [81]:
# kolon adlarını büyük harfe dönüştürüyorum.

df.columns = [col.upper() for col in df.columns]

In [82]:
# Müşteri yaşı, ürünlerin satış fiyatları ve bir yıl içerisinde satın alınma frekanslarına bir aralık vererek yeni değişkenler oluşturmak istiyorum.

# ilk olarak müşterilerin yaş aralıklarını kategorize edelim.

df.loc[(df['CUSTOMERAGE'] <= 25), 'NEW_SEX_CAT'] = 'YoungCustomer'
df.loc[(df['CUSTOMERAGE'] >= 25) & (df['CUSTOMERAGE'] < 50) , 'NEW_SEX_CAT'] = 'MatureCustomer'
df.loc[(df['CUSTOMERAGE'] >= 50), 'NEW_SEX_CAT'] = 'SeniorCustomer'

In [83]:
# Oluşturduğumuz yeni değişkeni satın alma niyetine oranla baktığımızda 
# yaşı büyük olan müşterilerin daha çok alışveriş yapma niyetinde oldukları gözlemlenmektedir.
# Bu saye de  güzel bir değişken türetmiş olduğumuzu söyleyebilirim.

df.groupby('NEW_SEX_CAT').agg({'PURCHASEINTENT':'mean'})

,PURCHASEINTENT
NEW_SEX_CAT,
MatureCustomer,0.574048
SeniorCustomer,0.677688
YoungCustomer,0.235804


In [84]:
# Ürünlerin fiyat aralıklarını kategorize etmeden önce ilk olarak en ucuz  ve en pahalı ürünün fiyatına bakmak istedim.

print(df['PRODUCTPRICE'].max())
print(df['PRODUCTPRICE'].min())
print(df['PRODUCTPRICE'].mean())

2999.852253399973
100.3763582884275
1527.4291946773735


In [85]:
# Şimdide ürünlerin fiyat aralıklarını kategorize edelim.
# Ürünlerin fiyat kategorilerini de 3 kategoriye ayırmak istiyorum.


df.loc[(df['PRODUCTPRICE'] <= 500), 'NEW_PRODUCTPRICE_CAT'] = 'Affordable'
df.loc[(df['PRODUCTPRICE'] >= 500) & (df['PRODUCTPRICE'] < 2000) , 'NEW_PRODUCTPRICE_CAT'] = 'ALittleExpensive'
df.loc[(df['PRODUCTPRICE'] >= 2000), 'NEW_PRODUCTPRICE_CAT'] = 'Expensive'

In [86]:
# fiyat kategorilerine göre analiz yaptığımızda da orta pahalılıkta olan segmentlerin daha çok satın alınma potansiyeli olduğu gözlemleniyor.

df.groupby('NEW_PRODUCTPRICE_CAT').agg({'PURCHASEINTENT':'mean'})

,PURCHASEINTENT
NEW_PRODUCTPRICE_CAT,
ALittleExpensive,0.578125
Affordable,0.557861
Expensive,0.551644


In [87]:
# Şimdide yıllık satın alınma frekanslarına göre bir kategori oluşturmak istiyorum.
# yine ilk olarak en düşük ve en yüksek olan frekans aralıklarına bakıyorum. Şimdi buna göre 4 kategoriye ayıracağız.

print(df['PURCHASEFREQUENCY'].max())
print(df['PURCHASEFREQUENCY'].min())
print(df['PURCHASEFREQUENCY'].mean())

19
1
10.054666666666666


In [88]:
df.loc[(df['PURCHASEFREQUENCY'] >= 1) & (df['PURCHASEFREQUENCY'] < 5), 'NEW_PURCHASEFREQUENCY_CAT'] = 'OccasionallyPurchased'
df.loc[(df['PURCHASEFREQUENCY'] >= 5) & (df['PURCHASEFREQUENCY'] < 10) , 'NEW_PURCHASEFREQUENCY_CAT'] = 'SellingProduct'
df.loc[(df['PURCHASEFREQUENCY'] >= 10) & (df['PURCHASEFREQUENCY'] < 15) , 'NEW_PURCHASEFREQUENCY_CAT'] = 'BestSellingProduct'
df.loc[(df['PURCHASEFREQUENCY'] >= 15), 'NEW_PURCHASEFREQUENCY_CAT'] = 'ContinuouslySoldProducts'

In [89]:
# Her frekans aralığında olan ürünlerin neredeyse eşit miktarda satıldığını görmekteyiz.
# Satın alınma niyetine çok fazla katkısı olmasa da elimizde dursun

df.groupby('NEW_PURCHASEFREQUENCY_CAT').agg({'PURCHASEINTENT':'mean'})

,PURCHASEINTENT
NEW_PURCHASEFREQUENCY_CAT,
BestSellingProduct,0.562552
ContinuouslySoldProducts,0.568239
OccasionallyPurchased,0.576490
SellingProduct,0.560677


In [90]:
# Product ID makine öğrenmesinde modeli şaşırtabileceği için bu sütunu kaldırıyorum.

df.drop('PRODUCTID',axis=1, inplace=True)

In [91]:
df.head()

,PRODUCTCATEGORY,PRODUCTBRAND,PRODUCTPRICE,CUSTOMERAGE,CUSTOMERGENDER,PURCHASEFREQUENCY,CUSTOMERSATISFACTION,PURCHASEINTENT,NEW_SEX_CAT,NEW_PRODUCTPRICE_CAT,NEW_PURCHASEFREQUENCY_CAT
0,Smartphones,Other Brands,312.949668,18,0,2,1,0,YoungCustomer,Affordable,OccasionallyPurchased
1,Smart Watches,Samsung,980.389404,35,1,7,2,1,MatureCustomer,ALittleExpensive,SellingProduct
2,Tablets,Samsung,2606.718293,63,0,1,5,1,SeniorCustomer,Expensive,OccasionallyPurchased
3,Smartphones,Samsung,870.395450,63,1,10,3,1,SeniorCustomer,ALittleExpensive,BestSellingProduct
4,Tablets,Sony,1798.955875,57,0,17,3,0,SeniorCustomer,ALittleExpensive,ContinuouslySoldProducts


In [109]:
# En önemli hayat kurtaran fonksiyondur.

# Numerik görünümlü kategorik değişkenlerin bulunması için üst sınır değerini 10 veriyoruz.
# Kategorik ama Kardinal değerlerin üst sınırı için ise 20 değerini veriyoruz ve fonksiyonumuzu yazmaya geçiyoruz.
# Fonksiyonun ne anlama geldiğini başkalarına anlatmak için değerleri yazıyoruz.

def grab_col_names(dataframe, cat_th=10, car_th=20):
    
    """
    
    Verisetindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir
    
    Parameters
    -----------------
    
    dataframe: dataframe
        Değişken isimleri alınmak istenen dataframedir.
        
    cat_th: int, float
        numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        
    car_th: int, float
        kategorik fakat kardinal olan değişkenler için sınıf eşik değeri
    
    Returns
    -----------------
    
    cat_cols : list
        Kategorik değişken listesi
            
    num_cols : list
        Numerik değişken listesi
        
    cat_but_car : list
        Kategorik görünümlü kardinal değişken listesi
    
    Notes 
    
    -----------------
    
    cat_cols + num_cols + cat_but_car = toplam değişken sayısı
    
    num_but_cat cat_cols'un içerisinde.
            
    """
    
    # İlk olarak kategorik değişkenleri alıyoruz. Fonksiyonel olacağı için df leri dataframe
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['object','category','bool']]
    
    # Numerik görünümlü kategorik değişkenleri bulduğumuz yapıyı alıyoruz.
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < 10 and str(dataframe[col].dtypes) in ['int64','int32','float64']]
    
    # Kategorik değişkenleri hem cat cols hemde numerik ama kategorik olanları birleştirerek oluşturuyoruz.
    
    cat_cols = cat_cols + num_but_cat
   
    # kategorik fakat kardinal olan değerleri almak için ise, kolonlarda gez, eşsiz değeri 20 den büyük olan ve 
    # category ve object olan değişkenleri al diyoruz.
    
    # NOT : Bu seneryoda bu konuda bir değişken bulunmuyor fakat farklı bir verisetinde olabilir, o nedenle önemli bir bilgidir.
    
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique()>20 and str(dataframe[col].dtypes) in ['category','object']]
    
    # Eğer kategorik ama kardinal olan değişkenlerden de bir sonuç gelirse bunu cat colstan çıkarmamız gerekli.
    
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    
    # Numerik değişkenleri bulmak için ise ilk olarak df kolonlarında gez, dtype ı integer ve float olan değerleri al dedik.
    # Fakat numerik görünümlü kategorik değişkenler ile ortak değişkenler gelebileceği için ek bir kod yazdık.
    # bu kodta, numerik kolonlarda gez eğer categorik kolonlarda yoksa getir dedik.
    
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ['int32','int64','float64']]
    num_cols = [col for col in num_cols if col not in cat_cols]
    
    # Fonksiyonumuza bir raporlama bölümü ekliyoruz.
    
    # Raporlamada gözlem sayısını, değişkenleri, kategorik kolonların boyutunu, numerik kolonların boyutunu, 
    # kategorik fakat kardinal olanların boyutunu
    # ve numerik fakat kategorik olanların boyutunu yazdırıyoruz.
    
    print(f'GozlemSayısı:{dataframe.shape[0]}')
    print(f'Değişkenler:{dataframe.shape[1]}')
    print(f'cat_cols:{len(cat_cols)}')
    print(f'num_cols:{len(num_cols)}')
    print(f'cat_but_car:{len(cat_but_car)}')
    print(f'num_but_cat:{len(num_but_cat)}')
    
    return num_cols, cat_cols, cat_but_car

In [113]:
num_cols, cat_cols, cat_but_car = grab_col_names(df)

GozlemSayısı:9000
Değişkenler:11
cat_cols:8
num_cols:3
cat_but_car:0
num_but_cat:3


In [115]:
# Kategorik değişkenleri analiz etmek için kullandığımız fonksiyon.

def cat_summary(data, col):
    print(pd.DataFrame({col: data[col].value_counts(),
                       'ratio': 100* data[col].value_counts()/len(data[col]) }))
    print('\n')

In [117]:
for i in cat_cols:
    cat_summary(df,i)

                 PRODUCTCATEGORY      ratio
PRODUCTCATEGORY                            
Laptops                     1842  20.466667
Smartphones                 1841  20.455556
Smart Watches               1810  20.111111
Tablets                     1769  19.655556
Headphones                  1738  19.311111


              PRODUCTBRAND      ratio
PRODUCTBRAND                         
Samsung               1854  20.600000
HP                    1820  20.222222
Sony                  1790  19.888889
Other Brands          1776  19.733333
Apple                 1760  19.555556


                NEW_SEX_CAT      ratio
NEW_SEX_CAT                           
MatureCustomer         4254  47.266667
SeniorCustomer         3478  38.644444
YoungCustomer          1268  14.088889


                      NEW_PRODUCTPRICE_CAT      ratio
NEW_PRODUCTPRICE_CAT                                 
ALittleExpensive                      4736  52.622222
Expensive                             3011  33.455556
Affordabl

In [119]:
# Kategorik değişkenleri fonksiyonel olarak one hot encoding işlemi yapan kod

def one_hot_encoder(data, cat_cols, drop_first=True):
    data = pd.get_dummies(data, columns=cat_cols, drop_first=drop_first)
    data = data.astype(int)
    return data

In [121]:
# Bu kısımda sınıf sayısı 2 den büyük olan değişkenleri ele alarak onlara one hot encoding işlemi yapmak istediğimiz için
# gerekli kodu yazıyoruz. kodumuzda tüm kolonlarda gez 10 dan küçük ve 2 den büyük sınıfa sahip olanları seç dedim.

ohe_cols = [col for col in df.columns if 10 >=df[col].nunique() > 2]
ohe_cols

['PRODUCTCATEGORY',
 'PRODUCTBRAND',
 'CUSTOMERSATISFACTION',
 'NEW_SEX_CAT',
 'NEW_PRODUCTPRICE_CAT',
 'NEW_PURCHASEFREQUENCY_CAT']

In [123]:
df = one_hot_encoder(df,ohe_cols)

In [125]:
df.head()

,PRODUCTPRICE,CUSTOMERAGE,CUSTOMERGENDER,PURCHASEFREQUENCY,PURCHASEINTENT,PRODUCTCATEGORY_Laptops,PRODUCTCATEGORY_Smart Watches,PRODUCTCATEGORY_Smartphones,PRODUCTCATEGORY_Tablets,PRODUCTBRAND_HP,...,CUSTOMERSATISFACTION_3,CUSTOMERSATISFACTION_4,CUSTOMERSATISFACTION_5,NEW_SEX_CAT_SeniorCustomer,NEW_SEX_CAT_YoungCustomer,NEW_PRODUCTPRICE_CAT_Affordable,NEW_PRODUCTPRICE_CAT_Expensive,NEW_PURCHASEFREQUENCY_CAT_ContinuouslySoldProducts,NEW_PURCHASEFREQUENCY_CAT_OccasionallyPurchased,NEW_PURCHASEFREQUENCY_CAT_SellingProduct
0,312,18,0,2,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,1,0
1,980,35,1,7,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2606,63,0,1,1,0,0,0,1,0,...,0,0,1,1,0,0,1,0,1,0
3,870,63,1,10,1,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1798,57,0,17,0,0,0,0,1,0,...,1,0,0,1,0,0,0,1,0,0


In [127]:
df.shape

(9000, 24)

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 24 columns):
 #   Column                                              Non-Null Count  Dtype
---  ------                                              --------------  -----
 0   PRODUCTPRICE                                        9000 non-null   int32
 1   CUSTOMERAGE                                         9000 non-null   int32
 2   CUSTOMERGENDER                                      9000 non-null   int32
 3   PURCHASEFREQUENCY                                   9000 non-null   int32
 4   PURCHASEINTENT                                      9000 non-null   int32
 5   PRODUCTCATEGORY_Laptops                             9000 non-null   int32
 6   PRODUCTCATEGORY_Smart Watches                       9000 non-null   int32
 7   PRODUCTCATEGORY_Smartphones                         9000 non-null   int32
 8   PRODUCTCATEGORY_Tablets                             9000 non-null   int32
 9   PRODUCTBRAND_HP    

In [131]:
# Veri tanıma, veri ön işleme ve özellik mühendisliği işlemlerini tamamladıktan sonra 
# şimdi artık makine öğrenmesi projesi için model kurma işlemine geçebiliriz.

# Modeli kurmadan önce test ve eğitim setlerini x (bağımlı değişken) ve y(bağımsız değişkenler) olarak tanımlamamız gerekmektedir.
# x i bağımlı değişken olarak alacağımız için tahmin edeceğimiz müşterinin alışveriş yapma niyeti sütununu x değişkenine tanımlıyorum.

# sonrasında train_test_split fonksiyonuna x_train, x_test, y_train, y_test verilerini tanımlıyorum.

y = df['PURCHASEINTENT']
x = df.drop('PURCHASEINTENT',axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state=42)

In [133]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8100, 23)
(900, 23)
(8100,)
(900,)


In [135]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [137]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)
y_pred = rf_model.predict(x_test)
print('{:.2f}'.format(accuracy_score(y_pred,y_test)))

0.95


In [139]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(x_train,y_train)
k_pred = knn.predict(x_test)
print('{:.2f}'.format(accuracy_score(k_pred,y_test)))

0.56
